#Ładowanie zależności

In [ ]:
from PIL import Image
import cv2
import numpy as np
import math
import os
from numpy import loadtxt
import glob
import random
from pathlib import Path


#Porządkowanie zbioru zdjęć
Początkowo należy uporządkować zbiór zdjęć. Zdjęcia numerowane są kolejnymi liczbami naturalnymi, zmienimy im nazwę na taką, która zawiera epsilony- łatwiej takie zdjęcia się przegląda. Plik z epsilonami oraz zdjęcia pakujemy do data/external.

In [ ]:
lines=loadtxt('data/external/c211021.txt')
for i in range(lines.shape[0]):
    
    oldname='data/external/'+str(i).zfill(5)+'.png'
    newname='data/external/'+ str(format(lines[i,1].astype(float),'.3f')).replace('.','_')+'__'+str(i).zfill(5)+'.png'
    if (i%100==0) : print('o:', oldname,'   n: ', newname)
    
    os.rename(oldname,newname)

#Generowanie szumów
Kolejnym krokiem jest generowanie szumów. Generowane są na podstawie zbioru zdjęć wcześniej uporządkowanych. Poniżej kod na generowanie określonej liczby szumów na podstawie losowej liczby zdjec

In [79]:
NoiseCount=1000
MinimumFotoCount=50
MaximumFotoCount=150
ExternalsFolder=str(Path().resolve().parent)+"\\data\\external\\"
PathToNoise=str(Path().resolve().parent)+"\\data\\interim\\noise\\"
try:
    os.mkdir(PathToNoise, 0o666)
except OSError as error:
    a=1

ImageNames=[]
def GenerateOneNoiseRamdomChoice(fotoCountMin, fotoCountMax, noiseNumber, imageNames):
    fotoCount=random.randint(fotoCountMin, fotoCountMax)
    Combined=np.zeros((480,640))
            
    for i in range(fotoCount):
        randomNumber=random.randint(0,len(imageNames)-1)
        Read=np.array(cv2.imread(ExternalsFolder+imageNames[randomNumber]))
        Read=Read[4:484,4:644,0]
        ReadMean=np.mean(Read)
        Combined=(Combined+Read)#-ReadMean
        #if (i%100==0): print(i)
    Combined=Combined/fotoCount
    Combined=Combined.astype(np.uint8)

    Noise=Image.fromarray(Combined)
    Noise.save(PathToNoise+str(noiseNumber)+".png")

for filename in os.listdir(ExternalsFolder):
        #img = cv2.imread(os.path.join(dir,filename))
        #if img is not None:
        if(filename.__contains__(".png") ): ImageNames.append(filename)

for i in range(NoiseCount):
    GenerateOneNoiseRamdomChoice(MinimumFotoCount,MaximumFotoCount,(format(i/10000.0,'.4f')).replace(".",""), ImageNames)
    if (i%10==0): print("Made ",i,"noise")

Made  0 noise
Made  10 noise
Made  20 noise
Made  30 noise
Made  40 noise


KeyboardInterrupt: 

#Generowanie zdjęć z szumem

Najpierw funkcje pomocnicze, potem generator

In [81]:
def GenerateImage(w,h,middleX,middleY,ringsDifference,epsilon,pixelMax,pixelMin,pixelBrightnessMaxNoise, noiseFilesPaths):
  pixelMean=(pixelMax+pixelMin)/2
  pixelDiff=pixelMax-pixelMean
  pixelNoise=random.randint(-pixelBrightnessMaxNoise, pixelBrightnessMaxNoise)
  data=np.ones((h,w))
  data=data[:,:]*pixelMean
  for x in range (w):
    for y in range(h):
      data[y,x]=data[y,x]+(pixelDiff*math.cos( 2*math.pi*( epsilon+ ((pow((x-middleX)*2,2)+pow((y-middleY)*2,2)) /ringsDifference))))+pixelNoise
    #print("x: ",x,", y:  ",y,", data:  ",data[x,y], "liczba " , 2*math.pi*( ((pow((x-w/2)*2,2)+pow((y-h/2)*2,2)) /RingsDifference)))
  data=data.astype(np.uint8)
  return Noising(data, noiseFilesPaths)

def Noising(generated, noiseFilesPaths):
    randomNoiseInterator=random.randint(0,len(noiseFilesPaths)-1)
    noise=np.array(cv2.imread(noiseFilesPaths[randomNoiseInterator]))
    noise=noise[:,:,0]
    noiseMean=np.mean(noise)
    differ=-(noise-noiseMean)
    generated=generated[:,:]
    generated=generated-differ
    generated=np.clip(generated,0,255)
    generated=generated.astype(np.uint8)
    noised=Image.fromarray(generated)
    return noised
    #noised.save("NoisedImage.png")


ImagesToGenerateNumber=100
W = 640 #width
H = 480 #height
RingsDifference=W*W/6 #Difference in squared distance from center between succesive stripes
#57 600 #68 266
MiddleXMaxVariation=50 #in pixels
MiddleYMaxVariation=50 #in pisels
EpsilonMin=0
EpsilonMax=0.9999
PixelBrightnessMaxNoise=5 #in number, +- that value

PixelMaxUpperLimit=210  #upper limit max brightness of pixel, 8bit-> 0-255
PixelMaxLowerLimit=170  #lower limit max brightness of pixel, 8bit-> 0-255

PixelMinUpperLimit=120 #upper limit min brightness of pixe, 8bit-> 0-255
PixelMinLowerLimit=80 #lower limit min brightness of pixe, 8bit-> 0-255

MiddleX=W/2 #Middle of centre, X axis
MiddleY=H/2 #Middle of centre, Y axis
#epsilon=0 #set epsilon
#Epsilons=np.arange(1.0)/1000.0

GeneratedFolder=str(Path().resolve().parent)+"\\data\\generated\\"
PathToNoiseFolder=str(Path().resolve().parent)+"\\data\\interim\\noise\\"
NoiseFilesPaths=[]
#generating noises list
for filename in os.listdir(PathToNoiseFolder):
        #img = cv2.imread(os.path.join(dir,filename))
        #if img is not None:
        if(filename.__contains__(".png") ): NoiseFilesPaths.append(str(PathToNoiseFolder)+str(filename))


for i in range(ImagesToGenerateNumber):
    epsilon=random.randint(EpsilonMin*10000,EpsilonMax*10000)/10000.0
    middleXVariation=random.randint(-MiddleXMaxVariation,MiddleXMaxVariation)
    middleYVariation=random.randint(-MiddleYMaxVariation, MiddleYMaxVariation)
    pixelMax=random.randint(PixelMaxLowerLimit, PixelMaxUpperLimit)
    pixelMin=random.randint(PixelMinLowerLimit, PixelMinUpperLimit)

    middleX=MiddleX+middleXVariation
    middleY=MiddleY+middleYVariation
    #print(middleX)
    img=GenerateImage(W,H,middleX,middleY,RingsDifference,epsilon,pixelMax,pixelMin,PixelBrightnessMaxNoise, NoiseFilesPaths)

    img.save(GeneratedFolder+(format(i/10000.0,'.4f')).replace(".","")+"_"+(format(epsilon,'.4f')).replace(".","_")+".png")
    if(i%10==0): print("Generated",i, "image")



Generated 0 image
Generated 10 image
Generated 20 image
Generated 30 image
Generated 40 image
Generated 50 image
Generated 60 image
Generated 70 image
Generated 80 image
Generated 90 image
